In [1]:
with open("GowallaNYC","r") as file:
    gowalla = file.readlines()

gowalla[0]

'194108.0\t0\t5\t14\t-73.78280639649999\t40.6438845363\t-73.9472579956\t40.7657052487\n'

In [3]:
from collections import defaultdict
import random

random.seed(42)

num_pois = 10978


def process_prmedata_format(dataset=gowalla):
    # loading the relevant information for GMF : user, poi visited;
    users_pois = defaultdict(list)
    # with open("GowallaNYC_Formated","w") as file:
    # file.write("\t".join(c for c in new_line))
    # new_line = [arr[1],arr[2],"1"]
    for line in dataset:
        arr = line.split("\t")
        user = int(arr[1])
        if int(arr[2]) not in users_pois[user]:
            users_pois[user].append(int(arr[2]))

    # generating train and test samples (and val could be added here)
    train_users_pois = dict()
    validation_users_pois = dict()
    test_users_pois = dict()
    for user in users_pois.keys():
        random.shuffle(users_pois[user])
        train_users_pois[user] = users_pois[user][:int(
            len(users_pois[user]) * 0.8)]
        validation_users_pois[user] = users_pois[user][int(
            len(users_pois[user]) * 0.8): int(len(users_pois[user]) * 0.9)]
        test_users_pois[user] = users_pois[user][int(
            len(users_pois[user]) * 0.9):]


    
    
    with open("GowallaNYC.train.rating", "w") as train_file, open("GowallaNYC.test.rating", "w") as test_file, open("GowallaNYC.validation.rating", "w") as validation_file:
        for user in train_users_pois.keys():
            for poi in train_users_pois[user]:
                arr = [user, poi, 1]
                train_file.write("\t".join(str(c) for c in arr) + "\n")
            for poi in validation_users_pois[user]:
                arr = [user, poi, 1]
                validation_file.write("\t".join(str(c) for c in arr) + "\n")
            for poi in test_users_pois[user]:
                arr = [user, poi, 1]
                test_file.write("\t".join(str(c) for c in arr) + "\n")

    with open("GowallaNYC.train.negative", "w") as train_negative_file, open("GowallaNYC.test.negative", "w") as test_negative_file, open("GowallaNYC.validation.negative", "w") as validation_negative_file:
        for user in test_users_pois.keys():
            for poi in train_users_pois[user]:
                negatives = []
                for _ in range(99):
                    j = random.randint(0, num_pois - 1)
                    while j in test_users_pois[user] or j in train_users_pois[user] or j in validation_users_pois[user]:
                        j = random.randint(0, num_pois - 1)
                    negatives.append(j)
                negatives_str = "\t".join(str(n) for n in negatives)
                line = "(" + str(user) + "," + str(poi)+")" + \
                    " \t " + negatives_str + "\n"
                train_negative_file.write(line)
            for poi in validation_users_pois[user]:
                negatives = []
                for _ in range(99):
                    j = random.randint(0, num_pois - 1)
                    while j in test_users_pois[user] or j in train_users_pois[user] or j in validation_users_pois[user]:
                        j = random.randint(0, num_pois - 1)
                    negatives.append(j)
                negatives_str = "\t".join(str(n) for n in negatives)
                line = "(" + str(user) + "," + str(poi)+")" + \
                    " \t " + negatives_str + "\n"
                validation_negative_file.write(line)
            for poi in test_users_pois[user]:
                negatives = []
                for _ in range(99):
                    j = random.randint(0, num_pois - 1)
                    while j in test_users_pois[user] or j in train_users_pois[user] or j in validation_users_pois[user]:
                        j = random.randint(0, num_pois - 1)
                    negatives.append(j)
                negatives_str = "\t".join(str(n) for n in negatives)
                line = "(" + str(user) + "," + str(poi) + ")" + \
                    " \t " + negatives_str + "\n"
                test_negative_file.write(line)


In [4]:
process_prmedata_format()